In [ ]:
import numpy as np
import pandas as pd
import copy
import random
import time
import pickle
import sys


from util.util_load             import read_txt, read_scenario
from util.util_reschedule       import random_events, store_schedule, update_schedule, snapshot
from util.util_display          import plot, pretty_table
from env_action.action_space    import action_space

maxtime                = 10
PopSize                = 100
critical_machines      = {5, 6, 7, 8, 9, 10, 11, 12, 13, 21, 22, 26, 27}
action_name            = ["GA", "TS", 
                          "LFOH", "LAPH", "LAP_LFO", 
                          "LFOH_TS", "LAPH_TS", "LFOH_GA", "LAPH_GA",
                          "CDR1", "CDR2", "CDR3", "CDR4", "CDR5", "CDR6",
                          "RCRS"]
K = 30

with open('SMALL/pickle_validation_instances_480.pkl', 'rb') as f:
    instances = pickle.load(f)
with open('SMALL/pickle_validation_scenarios_480.pkl', 'rb') as f:
    scenarios = pickle.load(f)

results    = []
ActionList = [2, 3, 4]
for action_id in ActionList:
    method = action_name[action_id]
    for instance_id in InstanceList:
        current_instance 	= instances[instance_id]
        for scenario_id in ScenarioList:
            current_Scenario = scenarios[scenario_id]

            J 					= current_instance.J
            I 					= current_instance.I
            X_ijk 				= current_instance.X_ijk
            S_ij 				= current_instance.S_ij
            C_ij 				= current_instance.C_ij
            C_j 				= current_instance.C_j
            p_ijk 				= current_instance.p_ijk
            h_ijk 				= current_instance.h_ijk
            d_j 				= current_instance.d_j
            n_j 				= current_instance.n_j
            MC_ji				= current_instance.MC_ji
            n_MC_ji 			= current_instance.n_MC_ji
            OperationPool 		= current_instance.OperationPool
            n_Mch               = np.sum(np.max(h_ijk, axis= (0, 1)))/K
            org_J               = copy.deepcopy(J)
            org_p_ijk      	    = copy.deepcopy(p_ijk)
            org_h_ijk      	    = copy.deepcopy(h_ijk)
            org_n_j        	    = copy.deepcopy(n_j)
            org_MC_ji      	    = copy.deepcopy(MC_ji)
            org_n_MC_ji    	    = copy.deepcopy(n_MC_ji)

            S_k                    = np.zeros((K))
            S_j                    = np.zeros((J))
            n_ops_left_j           = copy.deepcopy(n_j)

            t                      = 0
            JSet                   = list(range(J))
            OJSet                  = [[] for _ in range(J)]
            for j in JSet:
                OJSet[j]           = [i for i in range(int(n_j[j]))]

            events                 = {}
            MB_record              = {}

            # Running schedule
            completion = False
            # Random events
            print("random event")
            JA_event, MB_event, t, triggered_event, \
            affected_Oij, re, MB_record                           = random_events(t, J, K, X_ijk, S_ij, C_ij, C_j, JA_event, MB_event, MB_record)


            while completion == False and triggered_event is not None:
                if t < np.max(C_ij):
                    # Snapshot
                    S_k, S_j, J, I, JSet, OJSet, DSet, ODSet, OperationPool, \
                    n_ops_left_j, MC_ji, n_MC_ji, d_j, n_j, p_ijk, h_ijk,    \
                    org_p_ijk, org_h_ijk, org_n_j, org_MC_ji, org_n_MC_ji,   \
                    Oij_on_machine, X_ijk, S_ij, C_ij, C_j,                  \
                    JA_boolean, JA_long_boolean, JA_urgent_boolean,          \
                    MB_boolean, MB_critical_boolean, sum_re,                 \
                    CT_k, T_cur, Tard_job, NewJobList            = snapshot(t, triggered_event, MC_ji, n_MC_ji,                 \
                                                                            d_j, n_j, p_ijk, h_ijk, J, I, K, X_ijk, S_ij, C_ij, \
                                                                            OperationPool, re, S_k,                             \
                                                                            org_J, org_p_ijk, org_h_ijk, org_n_j,               \
                                                                            org_MC_ji, org_n_MC_ji, C_j                         )
                    # Store previous schedule
                    X_previous, S_previous, C_previous           = store_schedule(X_ijk, S_ij, C_ij)       
                    action_method 						         = action_space(J, I, K, p_ijk, h_ijk, d_j, n_j, 
                                                                                MC_ji, n_MC_ji, n_ops_left_j, OperationPool, S_k, S_j, 
                                                                                JSet, OJSet, Oij_on_machine, affected_Oij, 
                                                                                t, X_ijk, S_ij, C_ij, C_j, CT_k, T_cur, Tard_job,
                                                                                NewJobList, PopSize, maxtime)
                    
                    reschedule							         = action_method[action_id]
                    GBest, X_ijk, S_ij, C_ij, C_j                = reschedule()
                    print(GBest)
                    X_ijk, S_ij, C_ij, C_j                        = update_schedule(DSet, ODSet, X_ijk, S_ij, C_ij, X_previous, S_previous, C_previous)
                    
                    JA_event, MB_event, t, triggered_event,\
                    affected_Oij, re, MB_record                   = random_events(t, J, K, X_ijk, S_ij, C_ij, C_j, JA_event, MB_event, MB_record)
                else: completion = True

            tardiness = copy.deepcopy(GBest)
            results.append({
                    'Method'    : method,
                    'InstanceID': instance_id,
                    'ScenarioID': scenario_id,
                    'Tardiness' : tardiness
                    })